In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.normal(200,16,size = 100)

array([192.59484244, 218.96959811, 193.14337127, 198.10857267,
       202.42045016, 218.60141705, 195.0662032 , 186.62921133,
       214.25236428, 196.33952613, 206.22884164, 202.94973854,
       219.0213816 , 204.6841198 , 180.9684454 , 212.21906015,
       209.48117599, 189.07753063, 208.47317108, 179.41591907,
       227.76055862, 203.05636343, 227.8389122 , 224.6085235 ,
       221.51727889, 192.88326133, 184.58473055, 228.57730533,
       197.356352  , 192.0950693 , 191.05864048, 219.61437354,
       169.0455283 , 238.15031985, 199.62023923, 200.81945665,
       195.97921123, 194.23039546, 207.31246404, 178.38856587,
       192.44389848, 204.57378907, 197.52608615, 202.78562648,
       211.14159455, 194.44567921, 194.27061359, 172.06359401,
       215.53428972, 224.3145079 , 194.84559033, 196.4455587 ,
       228.82716165, 190.38082553, 180.24790388, 193.83253433,
       194.65584538, 193.47667679, 174.20766428, 221.18061425,
       212.17065724, 212.01201999, 219.53350424, 215.56

In [3]:
spread_prob_df = pd.concat([pd.read_csv('../spread_probability_top30/%d.csv'%i) for i in range(1,6)],ignore_index=True)
spread_prob_df['from'] = spread_prob_df['from'].apply(lambda x: '%012d'%x)
spread_prob_df['to'] = spread_prob_df['to'].apply(lambda x: '%012d'%x)
spread_prob_df = spread_prob_df[['from', 'to', 'prob']].reset_index(drop = True)

In [4]:
import random

In [5]:
random.seed(42)
np.random.seed(42)

In [6]:
len(spread_prob_df['from'].unique())

216733

In [14]:
pop_data = pd.read_csv('../src_data/usa_population_revise.csv', dtype ={'GeoId':str, 'Population':np.int64})

In [81]:
len(pop_data)

220334

In [82]:
pop_data

,GeoId,Population
0,010010201001,730
1,010010201002,1263
2,010010202001,835
3,010010202002,1124
4,010010203001,2774
...,...,...
220329,721537506011,883
220330,721537506012,2523
220331,721537506013,991
220332,721537506021,1577


In [15]:
starts = list(spread_prob_df['from'].unique())
random.shuffle(starts)
starts = starts[:5]

In [83]:
def init_counter():
    counter = pop_data.reset_index()
    N = len(pop_data)
    counter['day'] = [0] * N
    counter['susceptible'] = counter['Population']
    counter['infectious'] = [0] * N
    counter['recovered'] = [0] * N
    counter = counter[['day', 'GeoId', 'susceptible', 'infectious', 'recovered']].reset_index(drop = True)
    
    total_infected_tracker = pd.DataFrame()
    total_infected_tracker['GeoId'] = counter['GeoId']
    total_infected_tracker['Infected'] = 0
    total_infected_tracker = total_infected_tracker.set_index('GeoId')
    total_infected_tracker = total_infected_tracker.to_dict()['Infected']
    return counter, total_infected_tracker

In [174]:
num_init_cases = 500
infectious_rate = [0.1,0.2,0.3,0.3,0.2,0.1,0.1]
infectious_day = len(infectious_rate)
spread_prob_grouped_df = spread_prob_df.groupby('from')
active_case = {}
counter,total_infected_tracker = init_counter()
region_level_output_str = ''
case_level_output_str = ''
simulation_day = 0
# random initiate spread in 5 regions
for s in starts:
    s_idx = counter[counter['GeoId'] == s].index
    s_pop = counter.loc[s_idx,'susceptible'].item()
    num_init_cases_refine = min(s_pop, num_init_cases)
    # for test
    active_case[s] = [[0] * infectious_day, [0] * infectious_day] # [case/day, idx/day]
    active_case[s][0][0] = num_init_cases_refine
    counter.loc[s_idx,'susceptible'] -= num_init_cases_refine
    counter.loc[s_idx,'infectious'] += num_init_cases_refine
    total_infected_tracker[s] += num_init_cases_refine
    # region level output update, ignore header
    # header: Day, GeoId, Susceptible, Infectious, Recovered
    region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, s, counter.loc[s_idx,'susceptible'].item(), counter.loc[s_idx,'infectious'].item(), counter.loc[s_idx,'recovered'].item())

    # case level output update, ignore header
    # header: Day, from_GeoId, from_case, to_GeoId, to_case
    # simulation initialed: 000..0, -1
    simu_GeoId = '0' * len(s)
    for i in range(num_init_cases_refine):
        case_level_output_str += '%d, %s, %d, %s, %d\n'%(simulation_day,simu_GeoId, -1, s, i)

# save the outputs
# Todo

In [17]:
def get_random_des(des_prob_df):
    rand_val = random.random()
    cumprob = des_prob_df['prob'].cumsum()
    for des, prob in zip(des_prob_df['to'], cumprob):
        if rand_val < prob: return des
    return des_prob_df.loc[-1,'to']

In [173]:
def next_day(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    new_cases = {}
    for cbg in list(active_case.keys()):
        
        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        cbg_active_case = active_case[cbg][0]
        cbg_case_idx_lower = active_case[cbg][1]
        src_cbg_idx = counter[counter['GeoId'] == cbg].index
        print(cbg_active_case)
        for d in range(infectious_day-1, -1, -1):

            num_infectious = cbg_active_case[d]

            # update for recovered cases
            if d == infectious_day-1:
                print (cbg_active_case[d])
                counter.loc[src_cbg_idx, 'recovered'] += cbg_active_case[d]
                counter.loc[src_cbg_idx, 'infectious'] -= cbg_active_case[d]
                # update region level output on recovered cases
                region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, cbg,
                                                                   counter.loc[src_cbg_idx,'susceptible'].item(),
                                                                   counter.loc[src_cbg_idx,'infectious'].item(),
                                                                   counter.loc[src_cbg_idx,'recovered'].item())
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
            else:
                cbg_active_case[d] = 0
            
            if num_infectious == 0: continue
                
            # calculate number of new cases in infectious day 'd'
            num_new_cases = 0
            if num_infectious < 30:
                for case_idx in range(num_infectious): 
                    if random.random() < infectious_rate[d]: num_new_cases += 1
            else:
                expected_cases = infectious_rate[d]*num_infectious
                if expected_cases >= 5:
                    num_new_cases = int(np.floor(max(np.random.normal(expected_cases,expected_cases*(1-infectious_rate[d])), 0)))
                else:
                    num_new_cases = np.random.poisson(expected_cases)
            if num_new_cases == 0: continue

            # calculate number of new cases in each possible infected cbg
            new_cases_by_infectious_day = {}
            num_new_cases_actual = 0
            if num_new_cases < 30: # Number of cases too small to use statistical methods
                for i in range(num_new_cases):
                    des_cbg = get_random_des(des_prob_df)
                    # Spread to a fully infected region, assume nothing happens
                    if counter[counter['GeoId'] == des_cbg]['susceptible'].item() == 0: continue
                    if des_cbg not in new_cases_by_infectious_day: new_cases_by_infectious_day[des_cbg] = 0
                    new_cases_by_infectious_day[des_cbg] += 1
                    num_new_cases_actual += 1
            else: # Approximate Binomial distribution with Normal distribution and Poisson distribution
                for i in des_prob_df.index:
                    des = des_prob_df.loc[i, 'to']
                    des_s_num = int(counter[counter['GeoId'] == des]['susceptible'].item())
                    if des_s_num == 0: continue
                    
                    prob = des_prob_df.loc[i, 'prob']
                    expected_cases = num_new_cases * prob
                    expected_no_cases =  num_new_cases * (1-prob)
                    if expected_cases > 5 and expected_no_cases > 5:
                        var_case = expected_cases * (1-prob)
                        des_new_case = int(np.floor(np.random.normal(expected_cases,var_case)))
                    else:
                        des_new_case = np.random.poisson(expected_cases)
                        
                    # Check if fully infected
                    des_new_case = min(des_s_num, des_new_case)
                    des_new_case = max(0,des_new_case)
                    if des_new_case > 0: new_cases_by_infectious_day[des] = des_new_case
                    num_new_cases_actual += des_new_case
            src_case_id = cbg_case_idx_lower[d] + np.random.choice(num_infectious, size = num_new_cases_actual, replace = False)
            
            # update total `new_cases` counter
            count_output_num = 0
            for des,des_num_case in new_cases_by_infectious_day.items():
                if des not in new_cases: new_cases[des] = 0
                prev_num = new_cases[des]
                new_cases[des] += des_num_case

                # update case level output
                for i in range(des_num_case):
                    case_level_output_str += '%d, %s, %d, %s, %d\n'%(simulation_day, cbg, src_case_id[count_output_num+i], des, prev_num + i)
                count_output_num += des_num_case


            # update for recovered cases
            if d == infectious_day-1:
                print (cbg_active_case[d])
                counter.loc[src_cbg_idx, 'recovered'] += cbg_active_case[d]
                counter.loc[src_cbg_idx, 'infectious'] -= cbg_active_case[d]
                cbg_active_case[0] = 0
                # update region level output on recovered cases
                region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, cbg,
                                                                   counter.loc[src_cbg_idx,'susceptible'].item(),
                                                                   counter.loc[src_cbg_idx,'infectious'].item(),
                                                                   counter.loc[src_cbg_idx,'recovered'].item())
            
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
                
        
        # Clear the cbg from activate case tracker if no case active
        if sum(cbg_active_case) == 0: active_case.pop(cbg)
        
    # update region level output on recovered cases  
            
    # update active case
    for des,num_new_cases in new_cases.items():
        if des not in active_case: 
            active_case[des] = [[0] * infectious_day, [total_infected_tracker[des]] * infectious_day]
        active_case[des][0][0] = num_new_cases
        active_case[des][1][0] = total_infected_tracker[des]
        
        des_cbg_idx = counter[counter['GeoId'] == des].index
        counter.loc[des_cbg_idx, 'susceptible'] -= num_new_cases
        counter.loc[des_cbg_idx, 'infectious'] += num_new_cases
        total_infected_tracker[des] += num_new_cases
        
        # update region level output
        region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, des,
                                                           counter.loc[des_cbg_idx,'susceptible'].item(),
                                                           counter.loc[des_cbg_idx,'infectious'].item(),
                                                           counter.loc[des_cbg_idx,'recovered'].item())
    return region_level_output_str, case_level_output_str

In [175]:
for i in range(7):
    next_day(i)

[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[0, 500, 0, 0, 0, 0, 0]
0
[1, 500, 0, 0, 0, 0, 0]
0
[5, 500, 0, 0, 0, 0, 0]
0
[0, 500, 0, 0, 0, 0, 0]
0
[3, 500, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[55, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[3, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[3, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 

In [176]:
counter

,day,GeoId,susceptible,infectious,recovered
0,0,010010201001,730,0,0
1,0,010010201002,1263,0,0
2,0,010010202001,835,0,0
3,0,010010202002,1124,0,0
4,0,010010203001,2774,0,0
...,...,...,...,...,...
220329,0,721537506011,883,0,0
220330,0,721537506012,2523,0,0
220331,0,721537506013,991,0,0
220332,0,721537506021,1577,0,0


In [56]:
for k,v in active_case.items():
    print(k, v[0], v[1])

220450309003 [1, 0, 0, 0, 0, 0, 0] [71353    517
Name: Infected, dtype: int64, 0, 0, 0, 0, 0, 0]
261251605002 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
420270115022 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
121270832082 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
180890413021 [14, 0, 0, 0, 0, 0, 0] [71353    570
Name: Infected, dtype: int64, 0, 0, 0, 0, 0, 0]
220450311002 [1, 0, 0, 0, 0, 0, 0] [71353    689
Name: Infected, dtype: int64, 500, 500, 500, 500, 500, 500]
220450302004 [3, 0, 0, 0, 0, 0, 0] [71353    503
Name: Infected, dtype: int64, 503, 503, 503, 503, 503, 503]
220450309002 [1, 0, 0, 0, 0, 0, 0] [71353    506
Name: Infected, dtype: int64, 506, 506, 506, 506, 506, 506]
220450303022 [1, 0, 0, 0, 0, 0, 0] [71353    507
Name: Infected, dtype: int64, 507, 507, 507, 507, 507, 507]
220450304005 [1, 0, 0, 0, 0, 0, 0] [71353    690
Name: Infected, dtype: int64, 508, 508, 508, 508, 508, 508]
220450305003 [1, 0, 0, 0, 0, 0, 0] [71353    510
Name: Infected, dtype: int64, 510,

In [50]:
print(case_level_output_str)

0, 000000000000, -1, 220450309003, 0
0, 000000000000, -1, 220450309003, 1
0, 000000000000, -1, 220450309003, 2
0, 000000000000, -1, 220450309003, 3
0, 000000000000, -1, 220450309003, 4
0, 000000000000, -1, 220450309003, 5
0, 000000000000, -1, 220450309003, 6
0, 000000000000, -1, 220450309003, 7
0, 000000000000, -1, 220450309003, 8
0, 000000000000, -1, 220450309003, 9
0, 000000000000, -1, 220450309003, 10
0, 000000000000, -1, 220450309003, 11
0, 000000000000, -1, 220450309003, 12
0, 000000000000, -1, 220450309003, 13
0, 000000000000, -1, 220450309003, 14
0, 000000000000, -1, 220450309003, 15
0, 000000000000, -1, 220450309003, 16
0, 000000000000, -1, 220450309003, 17
0, 000000000000, -1, 220450309003, 18
0, 000000000000, -1, 220450309003, 19
0, 000000000000, -1, 220450309003, 20
0, 000000000000, -1, 220450309003, 21
0, 000000000000, -1, 220450309003, 22
0, 000000000000, -1, 220450309003, 23
0, 000000000000, -1, 220450309003, 24
0, 000000000000, -1, 220450309003, 25
0, 000000000000, -1, 2

In [130]:
def next_day(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    new_cases = {}
    for cbg in list(active_case.keys()):
        
        # calculate number of new cases
        num_new_cases = 0
        
        for i in range(infectious_day-1, -1, -1):
            mean_num_case = infectious_rate[i]*active_case[cbg][i]
            if mean_num_case != 0: num_new_cases +=  max(np.random.normal(mean_num_case,mean_num_case*(1-infectious_rate[i])), 0)
            # update active case
            active_case[cbg][i] = active_case[cbg][i-1] if i > 0 else 0
        num_new_cases = int(np.floor(num_new_cases))
        
        # calculate number of new cases in each possible infected cbg
        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        if num_new_cases < 30:
            
            
            
            for i in range(num_new_cases):
                des_cbg = get_random_des(des_prob_df)
                
                # Spread to a fully infected region, assume nothing happens
                if counter[counter['GeoId'] == des_cbg]['susceptible'] == 0: continue
                
                if des_cbg not in new_cases: new_cases[des_cbg] = 0
                new_cases[des_cbg] += 1
        else:
            for i in des_prob_df.index:
                des = des_prob_df.loc[i, 'to']
                prob = des_prob_df.loc[i, 'prob']
                expected_case = num_new_cases * prob
                expected_no_case =  num_new_cases * (1-prob)
                des_new_case = 0
                if expected_case > 5 and expected_no_case > 5:
                    var_case = expected_case * (1-prob)
                    des_new_case = np.floor(np.random.normal(expected_case,var_case))
                else:
                    des_new_case = np.random.poisson(expected_case)
                if des_new_case > 0:
                    if des not in new_cases: new_cases[des] = 0
                    new_cases[des] += des_new_case
        

    
    
    
    # update active case
    for des,num_new_case in new_cases.items():
        if des not in active_case: active_case[des] = np.zeros(infectious_day)
        active_case[des][0] = num_new_case
        
        # update region level output
        des_idx = counter[counter['GeoId'] == des]
        region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, des_idx, counter.loc[des_idx,'susceptible'].item(), counter.loc[des_idx,'infectious'].item(), counter.loc[des_idx,'recovered'].item())

In [72]:
next_day()

In [73]:
active_case

{'220450309003': array([  1., 100.,   0.,   0.,   0.,   0.,   0.]),
 '261251605002': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '420270115022': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '121270832082': array([  1., 100.,   0.,   0.,   0.,   0.,   0.]),
 '180890413021': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '220450302003': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450303023': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450303011': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450301001': array([3., 0., 0., 0., 0., 0., 0.]),
 '220450312001': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450302004': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450305003': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450306004': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450304005': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450311003': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450312002': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450310004': array([1., 0., 0., 0., 0., 0., 0.]),
 '2204503110

In [75]:
next_day()
active_case

{'220450309003': array([  1.,   0.,   1., 100.,   0.,   0.,   0.]),
 '261251605002': array([  0.,   0.,   0., 100.,   0.,   0.,   0.]),
 '420270115022': array([  1.,   0.,   0., 100.,   0.,   0.,   0.]),
 '121270832082': array([  1.,   0.,   1., 100.,   0.,   0.,   0.]),
 '180890413021': array([  0.,   0.,   0., 100.,   0.,   0.,   0.]),
 '220450302003': array([0., 1., 1., 0., 0., 0., 0.]),
 '220450303023': array([0., 1., 2., 0., 0., 0., 0.]),
 '220450303011': array([3., 1., 1., 0., 0., 0., 0.]),
 '220450301001': array([3., 1., 3., 0., 0., 0., 0.]),
 '220450312001': array([5., 1., 2., 0., 0., 0., 0.]),
 '220450302004': array([2., 0., 1., 0., 0., 0., 0.]),
 '220450305003': array([1., 2., 1., 0., 0., 0., 0.]),
 '220450306004': array([3., 2., 1., 0., 0., 0., 0.]),
 '220450304005': array([2., 2., 1., 0., 0., 0., 0.]),
 '220450311003': array([0., 1., 2., 0., 0., 0., 0.]),
 '220450312002': array([2., 0., 1., 0., 0., 0., 0.]),
 '220450310004': array([3., 1., 1., 0., 0., 0., 0.]),
 '2204503110